In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
import seaborn as sns
import os, string, random, time, math
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import re
import string

device_gpu = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"
print(device_gpu)

/CPU:0


In [3]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU Name:", gpus[0].name)
else:
    print("No GPU found.")


Num GPUs Available: 0
No GPU found.


### Encoding

In [35]:
eng_alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
pad = '-PAD-'
sos = '^'
eos = '$'
engdic = {pad: 0, sos: 1, eos: 2}
for index ,alpha in enumerate(eng_alphabets):
  engdic[alpha] = index+3
print(engdic)

{'-PAD-': 0, '^': 1, '$': 2, 'A': 3, 'B': 4, 'C': 5, 'D': 6, 'E': 7, 'F': 8, 'G': 9, 'H': 10, 'I': 11, 'J': 12, 'K': 13, 'L': 14, 'M': 15, 'N': 16, 'O': 17, 'P': 18, 'Q': 19, 'R': 20, 'S': 21, 'T': 22, 'U': 23, 'V': 24, 'W': 25, 'X': 26, 'Y': 27, 'Z': 28}


In [36]:
#Telugu unicode for hex range is 3072:3199
tel_alphabets = [chr(alpha) for alpha in range(3072,3199)]
tel_alphabets.append(chr(8204))
tel_alphabets.append(chr(8205))
tel_size = len(tel_alphabets)
teldic = {pad : 0}
for index ,alpha in enumerate(tel_alphabets):
  teldic[alpha] = index+1
print(teldic)


{'-PAD-': 0, 'ఀ': 1, 'ఁ': 2, 'ం': 3, 'ః': 4, 'ఄ': 5, 'అ': 6, 'ఆ': 7, 'ఇ': 8, 'ఈ': 9, 'ఉ': 10, 'ఊ': 11, 'ఋ': 12, 'ఌ': 13, '\u0c0d': 14, 'ఎ': 15, 'ఏ': 16, 'ఐ': 17, '\u0c11': 18, 'ఒ': 19, 'ఓ': 20, 'ఔ': 21, 'క': 22, 'ఖ': 23, 'గ': 24, 'ఘ': 25, 'ఙ': 26, 'చ': 27, 'ఛ': 28, 'జ': 29, 'ఝ': 30, 'ఞ': 31, 'ట': 32, 'ఠ': 33, 'డ': 34, 'ఢ': 35, 'ణ': 36, 'త': 37, 'థ': 38, 'ద': 39, 'ధ': 40, 'న': 41, '\u0c29': 42, 'ప': 43, 'ఫ': 44, 'బ': 45, 'భ': 46, 'మ': 47, 'య': 48, 'ర': 49, 'ఱ': 50, 'ల': 51, 'ళ': 52, 'ఴ': 53, 'వ': 54, 'శ': 55, 'ష': 56, 'స': 57, 'హ': 58, '\u0c3a': 59, '\u0c3b': 60, '఼': 61, 'ఽ': 62, 'ా': 63, 'ి': 64, 'ీ': 65, 'ు': 66, 'ూ': 67, 'ృ': 68, 'ౄ': 69, '\u0c45': 70, 'ె': 71, 'ే': 72, 'ై': 73, '\u0c49': 74, 'ొ': 75, 'ో': 76, 'ౌ': 77, '్': 78, '\u0c4e': 79, '\u0c4f': 80, '\u0c50': 81, '\u0c51': 82, '\u0c52': 83, '\u0c53': 84, '\u0c54': 85, 'ౕ': 86, 'ౖ': 87, '\u0c57': 88, 'ౘ': 89, 'ౙ': 90, 'ౚ': 91, '\u0c5b': 92, '\u0c5c': 93, 'ౝ': 94, '\u0c5e': 95, '\u0c5f': 96, 'ౠ': 97, 'ౡ': 98, 'ౢ': 99, 'ౣ': 100, 

In [ ]:
print(tel_alphabets[6]+tel_alphabets[2])

ఆం


### preprocessing

In [37]:
X=[]
Y=[]
with open(r"EngToTel.txt",encoding = "utf8") as f:
  for line in f:
    line = line.split()
    eng = line[0].strip().upper()
    tel = line[1].strip()
    X.append(eng)
    Y.append(tel)

In [38]:
print(X[1])
print(Y[1])

DEVATA
దేవత


In [39]:
def checkAdjChar(instring):
    for i in range(0, len(instring)-2):
        if (instring[i] == instring[i+1]):
            return 1
        else:
            continue
    return 0


In [40]:
L=[]
M=[]
with open(r"TelToEng.txt",encoding = "utf8") as k:
    for line in k:
        line = line.split()
        EN = line[1].strip().upper()
        TE = line[0].strip()
        if(len(TE)>10 and checkAdjChar(EN)==0):
            L.append(EN)
            M.append(TE)

In [41]:
print(L[5])
print(M[5])

ANUMATINCHALEDU
అనుమతించలేదు


In [42]:
X = X+L
Y = Y+M

In [43]:

X=[f"^{word.upper()}$" for word in X]


In [44]:
print(X[1])

^DEVATA$


In [45]:
print(len(X))

16026


In [46]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [47]:
print(f"Training size:",len(X_train))
print(f"Test size:",len(X_test))

Training size: 12820
Test size: 3206


In [48]:
for i in range(3):
  print(X_train[i],Y_train[i])

^MANIK$ మాణిక్
^ASIMA$ అసీమా
^HALEEM$ హలీమ్


### **dataloading**

In [ ]:
class TFDataloader:
    def __init__(self, a, b):
        self.tel_words, self.eng_words = a, b
        self.shuffle_indices = list(range(len(self.tel_words)))
        random.shuffle(self.shuffle_indices)
        self.shuffle_start_index = 0

    def __len__(self):
        return len(self.tel_words)

    def __getitem__(self, idx):
        return self.tel_words[idx], self.eng_words[idx]

    def get_random_sample(self):
        idx = np.random.randint(len(self.tel_words))
        return self.__getitem__(idx)

    def get_batch_from_array(self, batch_size, array):
        end = self.shuffle_start_index + batch_size
        batch = []
        if end >= len(self.tel_words):
            batch = [array[i] for i in self.shuffle_indices[0:end % len(self.tel_words)]]
            end = len(self.tel_words)
        return batch + [array[i] for i in self.shuffle_indices[self.shuffle_start_index:end]]

    def get_batch(self, batch_size, postprocess=True):
        tel_batch = self.get_batch_from_array(batch_size, self.tel_words)
        eng_batch = self.get_batch_from_array(batch_size, self.eng_words)
        self.shuffle_start_index += batch_size

        if self.shuffle_start_index >= len(self.tel_words):
            random.shuffle(self.shuffle_indices)
            self.shuffle_start_index = 0

        return tel_batch, eng_batch

    def to_tf_dataset(self, batch_size):

        def batch_generator():
            while True:
                tel_batch, eng_batch = self.get_batch(batch_size)
                for tel, eng in zip(tel_batch, eng_batch):
                    yield tel, eng

        dataset = tf.data.Dataset.from_generator(
            batch_generator,
            output_signature=(
                tf.TensorSpec(shape=(), dtype=tf.string),
                tf.TensorSpec(shape=(), dtype=tf.string)
            )
        )
        return dataset.batch(batch_size)


In [ ]:

train_data = TFDataloader(Y_train, X_train)
test_data = TFDataloader(Y_test, X_test)

print(type(test_data))

for i in range(3):
    print(test_data[i])

<class '__main__.TFDataloader'>
('ప్రోటిస్టా', '^PROTIST$')
('శాండ్\u200cవిచ్', '^SANDWICH$')
('కపూర్', '^KAPOOR$')


In [51]:
bss=train_data.get_batch(10)
print(bss)

(['చోలా', 'శోభా', 'వెస్టిండీస్తో', 'మేర్లపాక', 'టాగూర్', 'పొట్లదుర్తి', 'ప్రధాన', 'ఇద్దరమ్మాయిలతో', 'రౌలింగ్', 'సుబ్రహ్మణ్య'], ['^CHOLA$', '^SOBHA$', '^WESTINDESTHO$', '^MERLAPAKA$', '^TAGORE$', '^POTLADURTHI$', '^PRADHAN$', '^IDDARAMMAYILATHO$', '^ROWLING$', '^SUBRAMANYA$'])


In [52]:
def word_rep_tf(word, dic):
    word = word.upper()
    vocab_size = len(dic)
    seq_len = len(word) + 1
    one_hot = tf.zeros((seq_len, vocab_size), dtype=tf.float32)

    for i, letter in enumerate(word):
        pos = dic[letter]
        one_hot = tf.tensor_scatter_nd_update(one_hot, [[i, pos]], [1.0])

    pad_pos = dic[pad]
    one_hot = tf.tensor_scatter_nd_update(one_hot, [[len(word), pad_pos]], [1.0])
    return tf.expand_dims(one_hot, axis=0)  # Shape: (1, seq_len, vocab_size)


In [53]:
def gt_rep(word, dic, device='/CPU:0'):
    word = word.upper()
    with tf.device(device):
        rep = tf.zeros([len(word) + 1, 1], dtype=tf.int64)

        indices = []
        updates = []

        for letter_index, letter in enumerate(word):
            pos = dic[letter]
            indices.append([letter_index, 0])
            updates.append(pos)

        # Add padding at the end
        indices.append([len(word), 0])
        updates.append(dic[pad])

        rep = tf.tensor_scatter_nd_update(rep, indices=indices, updates=updates)
        rep = tf.reshape(rep, (1, -1))

    return rep

In [32]:
for i in range(3):
    print(Y[i])

పంజాబీ
దేవత
బెషెవిస్


In [54]:
a=word_rep_tf(Y[0],teldic)

print(a.shape)

(1, 7, 130)


In [55]:
tel_batch, eng_batch = train_data.get_batch(32)
for i in range(len(tel_batch)):
  print(tel_batch[i],eng_batch[i])
  input_seq = word_rep_tf(tel_batch[i], teldic)  # shape: (1, seq_len, tel_vocab_size)
  gt_seq = gt_rep(eng_batch[i], engdic)
  print(input_seq.shape)
  print(gt_seq.shape)

చన్నపట్న ^CHANNAPATNA$
(1, 9, 130)
(1, 14)
పెర్సల్ఫేట్ ^PERSULFATE$
(1, 12, 130)
(1, 13)
నాగార్జున ^NAGARJUN$
(1, 10, 130)
(1, 11)
రీమాన్ ^RIEMANN$
(1, 7, 130)
(1, 10)
భార్గవి ^BHARGAVI$
(1, 8, 130)
(1, 11)
వాసుదేవ్ ^VASUDEV$
(1, 9, 130)
(1, 10)
దిగువమాసపల్లె ^DIGUVAMASAPALLE$
(1, 14, 130)
(1, 18)
డికెన్స్ ^DICKENS$
(1, 9, 130)
(1, 10)
కార్డ్ ^CARD$
(1, 7, 130)
(1, 7)
కమ్యూనిటీ ^COMMUNITY$
(1, 10, 130)
(1, 12)
నందవరం ^NANDAVARAM$
(1, 7, 130)
(1, 13)
కోయాజీ ^COYAJI$
(1, 7, 130)
(1, 9)
లక్సెట్టిపేట ^LUXETTIPET$
(1, 13, 130)
(1, 13)
హలా ^HALA$
(1, 4, 130)
(1, 7)
వికీబుక్స్ ^WIKIBOOKS$
(1, 11, 130)
(1, 12)
సార్కాప్టిస్ ^SARCOPTES$
(1, 13, 130)
(1, 12)
మోర్స్ ^MORSE$
(1, 7, 130)
(1, 8)
అంటోన్ ^ANTON$
(1, 7, 130)
(1, 8)
నరనారాయణులు ^NARANARAYANA$
(1, 12, 130)
(1, 15)
స్వర్ణమాల్య ^SWARNAMALYA$
(1, 12, 130)
(1, 14)
ముంగిస ^MONGOOSE$
(1, 7, 130)
(1, 11)
పుదుచ్చేరి ^PUDUCHERRY$
(1, 11, 130)
(1, 13)
దీవించండి ^DEEVINCHANDI$
(1, 10, 130)
(1, 15)
నాటింగ్‌హామ్ ^NOTTINGHAM$
(1, 13, 130)
(1, 13)
స్మార

In [56]:
sample2 = gt_rep(X[0],engdic)
print(X[0])
print(sample2)

^PUNJABI$
tf.Tensor([[ 1 18 23 16 12  3  4 11  2  0]], shape=(1, 10), dtype=int64)


### **GRU With Attention**

In [ ]:
class TransliterationEncoderDecoder(tf.keras.Model):
    def __init__(self, input_size, hidden_size, output_size):
        super(TransliterationEncoderDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.encoder_rnn_cell = tf.keras.layers.GRU(hidden_size, return_sequences=True, return_state=True)
        self.decoder_rnn_cell = tf.keras.layers.GRU(hidden_size, return_state=True)

        self.h2o = tf.keras.layers.Dense(output_size, activation='softmax')
        self.U = tf.keras.layers.Dense(hidden_size)
        self.W = tf.keras.layers.Dense(hidden_size)
        self.attn = tf.keras.layers.Dense(1)
        self.out2hidden = tf.keras.layers.Dense(hidden_size)

    def call(self, input_, max_char=30, gt=None,training=True):
        batch_size = tf.shape(input_)[0]
        eout, hidden = self.encoder_rnn_cell(input_)
        decoder_state = hidden
        decoder_state = tf.reshape(decoder_state, (1,-1))
        decoder_input = tf.zeros((batch_size, 1, self.output_size)) 
        outputs = []
        U = self.U(eout)
        for i in range(max_char):
            W = self.W(decoder_state)
            W = tf.expand_dims(W, 1)
            attn_input = tf.nn.tanh(U + W)
            V = self.attn(attn_input)
            attn_weights = tf.nn.softmax(V, axis=1)
            attn_applied = tf.reduce_sum(attn_weights * eout, axis=1, keepdims=True)
            embedded = self.out2hidden(decoder_input)
            decoder_input_concat = tf.concat([embedded, attn_applied], axis=-1)
            out, decoder_state = self.decoder_rnn_cell(decoder_input_concat, initial_state=decoder_state)
            decoder_state = tf.reshape(decoder_state, (1,-1))
            out = self.h2o(tf.expand_dims(out, axis=1))
            outputs.append(out[:, 0, :])
            idx = tf.argmax(out, axis=2, output_type=tf.int32)

            if training and gt is not None:
                idx = tf.expand_dims(gt[:, i], axis=1)
            one_hot = tf.one_hot(idx, depth=self.output_size)

            decoder_input = tf.stop_gradient(one_hot)

        final_output = tf.stack(outputs, axis=1)
        return final_output






In [58]:
net = TransliterationEncoderDecoder(len(teldic),256,len(engdic))


In [ ]:
def infer(model, name, n, device='/CPU:0'):
    with tf.device(device):
        name_ohe = word_rep_tf(name, teldic)
        output_batch = model(name_ohe, max_char=n,training=False)
        output_single = output_batch[:, 0, :]

        return output_single

### **Training**

In [ ]:
def train_batch_tf(model, optimizer, loss_fn, tel_batch, eng_batch, teldic, engdic, max_char, device="/CPU:0"):
    with tf.device(device):
        batch_size = len(tel_batch)
        total_loss = 0

        for i in range(batch_size):
            input_seq = word_rep_tf(tel_batch[i], teldic)  
            gt_seq = gt_rep(eng_batch[i], engdic)
            gt_without_squeze=gt_seq     
            gt_seq = tf.squeeze(gt_seq, axis=0)  
            max_char = tf.shape(gt_seq)[0]
            with tf.GradientTape() as tape:
                output = model(input_seq, max_char=max_char,gt=gt_without_squeze)  
                output = output[0]  

                loss = 0

                for t in range(tf.shape(gt_seq)[0]):
                    token_loss = loss_fn(gt_seq[t], output[t])  
                    loss += token_loss
                loss = loss / tf.cast(tf.shape(gt_seq)[0], tf.float32)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            total_loss += loss.numpy()

        return total_loss / batch_size


In [ ]:
def train_setup_tf(model, optimizer, loss_fn, dataset, teldic, engdic, n_batches=100, batch_size=10, max_char=20, device="/CPU:0"):
    loss_arr = []

    for i in range(n_batches):
        tel_batch, eng_batch = dataset.get_batch(batch_size)
        loss = train_batch_tf(model, optimizer, loss_fn, tel_batch, eng_batch, teldic, engdic, max_char, device=device)
        loss_arr.append(loss)

        if True:
            print(f"Iteration {i}, Loss: {loss:.4f}")
            plt.plot(loss_arr, '-*')
            plt.xlabel("Iteration")
            plt.ylabel("Loss")
            plt.show()
            clear_output(wait=True)

    return loss_arr


In [ ]:
def calc_accuracy_tf(model, device='/CPU:0'):
    predictions = []
    accuracy = 0.0

    with tf.device(device):
        for i in range(len(test_data)):
            tel, eng = test_data[i]
            gt = gt_rep(eng, engdic, device)  

            outputs = infer(model, tel, gt.shape[0], device=device)  
            correct = 0

            for index, out in enumerate(outputs):
                pred_index = tf.argmax(out, axis=-1).numpy().flatten()[0]  
                true_index = gt[index][0].numpy()                   

                if pred_index == true_index:
                    correct += 1

            accuracy += correct / gt.shape[0]

        accuracy /= len(test_data)
        accuracy *= 100

        print(f"Test Accuracy: {accuracy:.2f}%")
        return accuracy

In [64]:
model = TransliterationEncoderDecoder(len(teldic), 256, len(engdic))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False,reduction=None)

In [65]:
train_setup_tf(model, optimizer,loss_fn,train_data,teldic,engdic, n_batches=500, batch_size=32, max_char=30, device=device_gpu)

[np.float32(2.9453077),
 np.float32(2.3512027),
 np.float32(2.3141294),
 np.float32(2.198294),
 np.float32(2.1011286),
 np.float32(2.0488281),
 np.float32(2.0700438),
 np.float32(1.9600104),
 np.float32(2.051372),
 np.float32(1.8274007),
 np.float32(2.0758634),
 np.float32(1.9404002),
 np.float32(1.8915738),
 np.float32(1.9109582),
 np.float32(1.7674453),
 np.float32(1.8158103),
 np.float32(1.8656394),
 np.float32(1.8245964),
 np.float32(1.7501571),
 np.float32(1.7516897),
 np.float32(1.6109655),
 np.float32(1.5609055),
 np.float32(1.5220327),
 np.float32(1.399371),
 np.float32(1.3807149),
 np.float32(1.3769822),
 np.float32(1.3955262),
 np.float32(1.3125676),
 np.float32(1.2005231),
 np.float32(1.3395296),
 np.float32(1.2318592),
 np.float32(1.2110763),
 np.float32(1.1712166),
 np.float32(1.0331424),
 np.float32(1.1416045),
 np.float32(1.0275171),
 np.float32(1.0712314),
 np.float32(0.9587411),
 np.float32(1.0150607),
 np.float32(0.87555575),
 np.float32(0.8199854),
 np.float32(0.9151

In [ ]:
current_accuracy = calc_accuracy_tf(model, device=device_gpu)
print("Test Accuracy:" ,current_accuracy)

In [ ]:

best_accuracy = -1.0
best_model_path = "best_model_tf.weights.h5"

model = TransliterationEncoderDecoder(len(teldic), 256, len(engdic))
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False,reduction=None)
for lr in [0.001]:
    for batch_size in [32, 64]:
        for n_batches in [500, , 200]:
            expt_id = f"{int(lr*100)}_{n_batches}_{batch_size}"
            train_setup_tf(model, optimizer,loss_fn,train_data,teldic,engdic, n_batches, batch_size, max_char=30, device=device_gpu)
            current_accuracy = calc_accuracy_tf(model, device=device_gpu)

            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy

                print(f"New best model saved with accuracy: {best_accuracy:.2f}%")

            tf.keras.backend.clear_session()


In [66]:
print(model.summary())

Model: "transliteration_encoder_decoder_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                     │ ((1, 12, 256), (1,     │       297,984 │
│                                 │ 256))                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ ((1, 256), (1, 256))   │       591,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (1, 1, 29)             │         7,453 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (1, 12, 256)           │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (1, 256)               │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (1, 12, 1)             │           257 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (1, 1, 256)            │         7,680 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,036,318 (3.95 MB)

 Trainable params: 1,036,318 (3.95 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
def test_one_example(net, tel_input, tel_dic, eng_dic, max_char=30):
    rev_eng_dic = {v: k for k, v in eng_dic.items()}

    input_seq = word_rep_tf(tel_input, tel_dic)
    output = net(input_seq, max_char=max_char, training=False)

    pred_indices = tf.argmax(output, axis=2)[0].numpy()
    pred_chars = [rev_eng_dic[i] for i in pred_indices]

    if pred_chars and pred_chars[0] == '^':
        pred_chars = pred_chars[1:]

    if '$' in pred_chars:
        stop_idx = pred_chars.index('$')
        pred_chars = pred_chars[:stop_idx]

    decoded = ''.join(pred_chars)
    print(f"Input: {tel_input} -> Predicted Output: {decoded}")


In [84]:
test_one_example(model, "ప్రతి", teldic, engdic, max_char=30)


Input: ప్రతి -> Predicted Output: PRATI


In [85]:
test_one_example(model, "పరమార్థం", teldic, engdic, max_char=30)


Input: పరమార్థం -> Predicted Output: PARAMARTHAM


In [86]:
test_one_example(model, "ప్రతిబింబించటానికి", teldic, engdic, max_char=30)


Input: ప్రతిబింబించటానికి -> Predicted Output: PRATIBINBINCHATINI


In [ ]:
def test_sentence_example(net, tel_sentence, tel_dic, eng_dic, max_char=30):
    rev_eng_dic = {v: k for k, v in eng_dic.items()}

    words = tel_sentence.strip().split()
    decoded_words = []

    for word in words:
        input_seq = word_rep_tf(word, tel_dic)
        output = net(input_seq, max_char=max_char, training=False)
        pred_indices = tf.argmax(output, axis=2)[0].numpy()
        pred_chars = [rev_eng_dic[i] for i in pred_indices]

        if pred_chars and pred_chars[0] == '^':
            pred_chars = pred_chars[1:]
        if '$' in pred_chars:
            pred_chars = pred_chars[:pred_chars.index('$')]

        decoded_word = ''.join(pred_chars)
        decoded_words.append(decoded_word)

    decoded_sentence = ' '.join(decoded_words)
    print(f"Input: {tel_sentence}")
    print(f"Predicted Output: {decoded_sentence}")


In [94]:
test_sentence_example(model, "దీన్ని ప్రయత్నించడానికి దిగువ మీ భాషను మరియు ఇన్‌పుట్ సాధనాన్ని ఎంచుకుని టైప్ చేయడాన్ని మొదలుపెట్టండి", teldic, engdic, max_char=30)


Input: దీన్ని ప్రయత్నించడానికి దిగువ మీ భాషను మరియు ఇన్‌పుట్ సాధనాన్ని ఎంచుకుని టైప్ చేయడాన్ని మొదలుపెట్టండి
Predicted Output: DEENNI PRAYATNINCHADANI DIGUVA MEE BHASHANU MARIA INPUT SADHANANNI ENCHUKUNI TIPE CHEYADANNI MODALUPETTINDI


In [98]:
test_sentence_example(model, "తెలుగు పదాలను ఇంగ్లీష్ అక్షరాలలోకి మార్పిడి చేసే ప్రాజెక్ట్", teldic, engdic, max_char=30)


Input: తెలుగు పదాలను ఇంగ్లీష్ అక్షరాలలోకి మార్పిడి చేసే ప్రాజెక్ట్
Predicted Output: TELUGU PADALANU INGLISH AKSHARALAKOLI MARPIDI CHESE PRAJECT


In [100]:
test_sentence_example(model, "ఈ ప్రాజెక్ట్ తెలుగు పదాలను శుద్ధంగా ఉచ్చరించేలా మారుస్తుంది", teldic, engdic, max_char=30)


Input: ఈ ప్రాజెక్ట్ తెలుగు పదాలను శుద్ధంగా ఉచ్చరించేలా మారుస్తుంది
Predicted Output: E PRAJECT TELUGU PADALANU SHUDDANGA UCHARINCHELA MARUSTUNDI


In [101]:
test_sentence_example(model, "పరిశోధనల ద్వారా అభివృద్ధి చెందిన ఈ యంత్రం ఖచ్చితమైన ఫలితాలు ఇస్తుంది", teldic, engdic, max_char=30)


Input: పరిశోధనల ద్వారా అభివృద్ధి చెందిన ఈ యంత్రం ఖచ్చితమైన ఫలితాలు ఇస్తుంది
Predicted Output: PARISHODHANALA DWARA ABHIVUDDHI CHENDINA E YANTRA KHACHITAMAINA FALITALU ISTUNDI
